In [6]:
pdf_path = "d:\\Users\\ShwetaS\\Downloads\\Application Form (Use & File) - IndiaFirst Life Guaranteed Protection Plus Plan v2.0_cir.pdf"
output_path = "D:\\Users\\ShwetaS\\Documents\\ETL training projects\\IndiaFirst ETL\\IndiaFirst_output.xlsx"

In [7]:
import pdfplumber
import pandas as pd
import re

In [ ]:
# Clean text
def clean_text(text):
    text = text.replace('√', '')                    # Remove checkmark
    return re.sub(r'\s+', ' ', text).strip()

# EXTRACT TEXT BETWEEN 6.2 and 6.3 
section_text = ""
found_6_2 = False
end_6_3 = False

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue
        lines = text.split('\n')

        # Skip first and last lines which are page header/footer
        if len(lines) > 2:
            lines = lines[1:-1]

        for line in lines:
            if not found_6_2 and '6.2' in line:
                found_6_2 = True
            if found_6_2:
                if '6.3' in line:
                    end_6_3 = True
                    break
                section_text += line + '\n'
        if end_6_3:
            break

# STRUCTURED HEADERS
lines = section_text.split('\n')
header_pattern = re.compile(r'^(\(*[a-zivxl]+\)*[\.\)])\s+(.*)', re.IGNORECASE)
data_dict = {}
current_header = None

for line in lines:
    line = clean_text(line.strip())
    if not line:
        continue

    header_match = header_pattern.match(line)
    if header_match:
        raw_header = header_match.group(2).strip()
        # Cut header at first ':' or '/'
        split_header = re.split(r'[:/]', raw_header, 1)
        current_header = split_header[0].strip()
        data_dict[current_header] = ""
    elif current_header:
        data_dict[current_header] += (line + "\n")

# FINAL DATAFRAME
structured_df = pd.DataFrame([data_dict])

structured_df.to_excel(output_path, index=False)
print(f"Excel saved: {output_path}")


Excel saved: D:\Users\ShwetaS\Documents\ETL training projects\IndiaFirst ETL\IndiaFirst_output.xlsx
